## Initial steps

* Mount the Google Drive
* Create images dir


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir images

## Serial Download

* Download one image per step
* Lower download

In [ ]:
import requests
import time

In [ ]:
# Endereço do arquivo de url das imagens de treino
# Troca por /content/drive/MyDrive/Image_segmentation/filtered_coco/url_val.txt
# Para acessar as imagens de validação

url_image = open('/content/drive/MyDrive/Image_segmentation/filtered_coco/url_train.txt', 'r')

url_lines = url_image.readlines()

In [ ]:
start = time.time()
for img_url in url_lines: 
  new_url = img_url[2:len(img_url)-3]
  name = new_url[len(new_url)-16:len(new_url)]

  resposta = requests.get(new_url, stream=True)

  if not resposta.ok:
    print("Ocorreu um erro, status:" , resposta.status_code)
  else:
    for dado in resposta.iter_content(1024):
      if not dado:
          break
      # Salva a imagem dentro do ambiente de execução
      saved_img = open('/content/images/' + name, 'wb') 
      saved_img.write(dado)
      saved_img.close()
    print("Imagem salva: ", name)
end = time.time()

In [ ]:
url_image.close()
name_image.close()

In [ ]:
print("[INFO] download took {:.6f} seconds".format(end - start))

## Multithreading Download

* Apply multithreading to a faster download 
* Based on project of [Hannibal Liang][]

[Hannibal Liang]: https://medium.com/analytics-vidhya/how-to-download-images-faster-using-multithreading-in-python-a31110468770

In [ ]:
import requests
import time
import concurrent.futures

In [ ]:
cont = 0
def download_image(img_url):
    global cont
    img_bytes = requests.get(img_url).content
    img_name = img_url[len(img_url)-16:len(img_url)]
    # Salva a imagem dentro do ambiente de execução
    with open('/content/images/' + img_name, 'wb') as img_file:
        img_file.write(img_bytes)
        cont += 1
        if cont%1000 == 0:
          # Essa linha pode resultar em IOPub data rate exceeded
          # Caso dê o erro, basta comentar o print
          print('Images downloaded: ', cont)

In [ ]:
# Endereço do arquivo de url das imagens de treino
# Troca por /content/drive/MyDrive/Image_segmentation/filtered_coco/url_val.txt
# Para acessar as imagens de validação

with open('/content/drive/MyDrive/Image_segmentation/filtered_coco/url_train.txt', 'r') as url_list:
    img_urls = [str(line[2:len(line)-3]) for line in url_list.readlines()]

In [ ]:
t1 = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:
    print('Images to download: ', len(img_urls))
    for url in img_urls:
        executor.submit(download_image, url)

t2 = time.perf_counter()

In [ ]:
print(f'Finished in {t2-t1} seconds')
print('Images downloaded are in /content/images/')